------------
# Importing Necessary Libraries

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import keras
import tensorflow as tf
import pickle

from tensorflow.keras import models, layers
from keras.callbacks import ModelCheckpoint
from tensorflow.python.client import device_lib
from sklearn.preprocessing import StandardScaler

from ipynb.fs.full.PreProcessing import get_moving_diff, rescaling, get_static_features

In [11]:
print(device_lib.list_local_devices())
config = tf.ConfigProto( device_count = {'GPU': 1} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1345165562400034908
]


---------------
# Loading Data into System

In [12]:
x_lstm_train = np.load(os.path.join('data', 'x_lstm_train.npy'))
x_lstm_test = np.load(os.path.join('data', 'x_lstm_test.npy'))

x_static_train = np.load(os.path.join('data', 'x_static_train.npy'))
x_static_test = np.load(os.path.join('data', 'x_static_test.npy'))

y_train = np.load(os.path.join('data', 'y_train.npy'))
y_test = np.load(os.path.join('data', 'y_test.npy'))

In [13]:
print(x_lstm_train.shape, x_lstm_test.shape)
print(x_static_train.shape, x_static_test.shape)
print(y_train.shape, y_test.shape)

(35776, 120) (8944, 120)
(35776, 12) (8944, 12)
(35776, 6) (8944, 6)


# Re-Structuring the data

In [14]:
x_lstm_train = np.expand_dims(x_lstm_train, axis=-1)
x_lstm_test = np.expand_dims(x_lstm_test, axis=-1)

x_static_train = np.expand_dims(x_static_train, axis=-1)
x_static_test = np.expand_dims(x_static_test, axis=-1)

In [15]:
print(x_lstm_train.shape, x_lstm_test.shape)
print(x_static_train.shape, x_static_test.shape)
print(y_train.shape, y_test.shape)

(35776, 120, 1) (8944, 120, 1)
(35776, 12, 1) (8944, 12, 1)
(35776, 6) (8944, 6)


# Model Creation

In [16]:
def r_square(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [17]:
# 32, 16, 30 
model = keras.Sequential()

lstm_input = layers.Input(shape=(x_lstm_train.shape[1], x_lstm_train.shape[2]))
static_input = layers.Input(shape=(x_static_train.shape[1], x_static_train.shape[2]))

lstm_stack_1, lstm_stack_1_c = layers.GRU(120, return_sequences=True, return_state=True)(lstm_input)
drop_lstm_1 = layers.Dropout(0.05)(lstm_stack_1)

lstm_stack_2, lstm_stack_2_c = layers.GRU(120, return_sequences=True, return_state=True)(drop_lstm_1)

static_dense = layers.Dense(2)(static_input)
static_flatten = layers.Flatten()(static_dense)

concatenate = layers.Concatenate()([lstm_stack_1_c, lstm_stack_2_c, static_flatten])


drop_2 = layers.Dropout(0.1)(concatenate)
dense_1 = layers.Dense(64)(drop_2)
drop_3 = layers.Dropout(0.05)(dense_1)
dense_2 = layers.Dense(16)(drop_3)
drop_4 = layers.Dropout(0.02)(dense_2)

output = layers.Dense(6)(drop_4)


model = models.Model(inputs=[lstm_input, static_input], outputs=output)
model.summary()
model.compile(
    loss='mse', 
    optimizer='adam', 
    metrics=[r_square,'mae']
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 120, 1)]     0                                            
__________________________________________________________________________________________________
gru (GRU)                       [(None, 120, 120), ( 43920       input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 12, 1)]      0                                            
__________________________________________________________________________________________________
dropout (Dropout)               (None, 120, 120)     0           gru[0][0]                        
_____________

In [ ]:
filepath = "model_gru_static-{epoch:03d}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='r_square', verbose=1,
    save_best_only=False, save_weights_only=True, mode='auto', period=1)

history = model.fit(
    [x_lstm_train, x_static_train], y_train,
    epochs=120,
    batch_size=48,
    verbose=1,
    validation_data=([x_lstm_test, x_static_test], y_test),
    shuffle=True,
    callbacks=[checkpoint]
)

In [ ]:
print(history.history.keys())

# Loss
plt.plot(history.history['loss'][1:])
plt.title('mse')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper right')
plt.show()

# Loss
plt.plot(history.history['r_square'][3:])
plt.title('r_square')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper right')
plt.show()

In [ ]:
model.save(os.path.join('data','model_gru_static.h5'))

# Testing Prediction sequence

In [18]:
cgm_df = pd.read_csv(os.path.join('data','CGMData.csv')).T

In [19]:
testing_data = cgm_df.T.iloc[:,-400:-1].to_numpy()
testing_data

array([[143., 145., 146., 146., 147., 149., 150., 155., 163., 167., 169.,
        170., 171., 173., 180., 187., 190., 182., 177., 173., 161., 153.,
        143., 140., 141., 139., 140., 142., 130., 110., 110., 112., 117.,
        118., 116., 115., 114., 114., 114., 114., 115., 115., 116., 117.,
        118., 119., 119., 120., 120., 121., 121., 120., 120., 120., 122.,
        123., 123., 123., 122., 123., 124., 125., 126., 126., 125., 124.,
        123., 123., 123., 122., 120., 117., 116., 116., 117., 118., 118.,
        119., 120., 121., 122., 124., 124., 124., 125., 125., 125., 125.,
        124., 122., 121., 119., 118., 119., 123., 132., 139., 142., 144.,
        146., 148., 152., 156., 158., 159., 159., 161., 166., 173., 181.,
        192., 202., 207., 208., 210., 212., 217., 221., 226., 231., 235.,
        239., 241., 244., 241., 239., 235., 228., 221., 213., 205., 199.,
        194., 185., 177., 171., 168., 167., 161., 154., 149., 150., 153.,
        154., 145., 135., 127., 123., 

In [20]:
model.load_weights('model_gru_static-110.h5')

In [ ]:
scaler = pickle.load((open(os.path.join('data','scaler.pkl'), 'rb')))

time_stamps = 100
ahead_pred = 48
org_lstm = testing_data[:,0:time_stamps]
y_ground = testing_data[:,time_stamps:time_stamps+ahead_pred]

y_pred = []
for i in range(ahead_pred):
#    input_lstm = get_moving_diff(org_lstm, replace_na=True)
    input_lstm = org_lstm
#    input_lstm = np.vectorize(rescaling)(input_lstm)
    
    input_static = get_static_features(org_lstm)
    input_static = scaler.transform(input_static)
    
    input_lstm = np.expand_dims(input_lstm, axis=-1)
    input_static = np.expand_dims(input_static, axis=-1)

    pred = model.predict([input_lstm, input_static])[0,0]
#     pred = rescaling(pred[0,0], newscale=(-100,100), oldscale=(-1,1))
#     pred = org_lstm[:,-1]+pred
#     if pred[0] > int(pred[0])+0.5:
#         pred[0] = int(pred[0])+1.0
#     else:
#         pred[0] = int(pred[0])
        
#     y_pred.append(pred)
#     org_lstm = np.concatenate((org_lstm[:,1:], np.expand_dims(pred, axis=0)), axis=-1)

#     pred = rescaling(org_lstm[:,-1], newscale=(0,1), oldscale=(20,450)) + pred
#     pred = rescaling(pred, newscale=(20,450), oldscale=(0,1))
    
#     if pred > int(pred)+0.5:
#         pred = int(pred)+1.0
#     else:
#         pred = int(pred)
    
#     pred = np.reshape(pred, (1,1))
#     y_pred.append(pred)
#     org_lstm = np.concatenate((org_lstm[:,1:], pred), axis=-1)

#    pred = org_lstm[:,-1] + pred
    
    if pred >= int(pred)+0.5:
        pred = int(pred)+1.0
    else:
        pred = int(pred)+0.0
    
    pred = np.reshape(pred, (1,1))
    y_pred.append(pred)
    org_lstm = np.concatenate((org_lstm[:,1:], pred), axis=-1)

In [21]:
scaler = pickle.load((open(os.path.join('data','scaler.pkl'), 'rb')))

time_stamps = 120
ahead_pred = 48
org_lstm = testing_data[:,0:time_stamps]
y_ground = testing_data[:,time_stamps:time_stamps+ahead_pred]

y_pred = []
for i in range(ahead_pred):
    input_lstm = get_moving_diff(org_lstm, replace_na=True)
    input_lstm = np.vectorize(rescaling)(input_lstm)
    
    input_static = get_static_features(org_lstm)
    input_static = scaler.transform(input_static)
    
    input_lstm = np.expand_dims(input_lstm, axis=-1)
    input_static = np.expand_dims(input_static, axis=-1)

    pred = model.predict([input_lstm, input_static])
    
    pred = pred[0][0]
    pred = rescaling(pred, newscale=(-100,100), oldscale=(-1,1))
    pred = org_lstm[:,-1]+pred
    
    if pred > int(pred)+0.5:
         pred = int(pred)+1.0
    else:
         pred = int(pred)+0.0
    
    pred = np.reshape(pred, (1,1))
    y_pred.append(pred)
    org_lstm = np.concatenate((org_lstm[:,1:], pred), axis=-1)

In [22]:
np.save('gru_static_good.npy',y_pred)